# Prometheus PromQL Quickstart
이 노트북은 포트포워딩된 Prometheus 엔드포인트를 대상으로 PromQL을 실행하고 결과를 판다스로 확인하는 예제입니다.

- `PROMETHEUS_URL` 환경 변수를 설정하면 기본 엔드포인트를 변경할 수 있습니다.
- `docs/study/prometheus/queries` 디렉터리에 `.promql` 파일을 저장해 두고 불러올 수 있습니다.


In [ ]:
import sys
import subprocess

def ensure_package(package):
    try:
        __import__(package)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

for pkg in ('requests', 'pandas'):
    ensure_package(pkg)


In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime
from pathlib import Path

PROMETHEUS_URL = os.getenv('PROMETHEUS_URL', 'http://localhost:9090')
print(f'Using Prometheus endpoint: {PROMETHEUS_URL}')


## Helper 함수
PromQL 쿼리를 실행하고 JSON 결과를 데이터프레임으로 변환합니다.

In [ ]:
def run_promql(query: str, when: datetime | None = None):
    params = {'query': query}
    if when is not None:
        params['time'] = when.timestamp()
    response = requests.get(f'{PROMETHEUS_URL}/api/v1/query', params=params, timeout=10)
    response.raise_for_status()
    payload = response.json()
    if payload.get('status') != 'success':
        raise RuntimeError(f"Prometheus query failed: {payload}")
    return payload['data']['result']

def promql_to_dataframe(result):
    records = []
    for item in result:
        metric = item.get('metric', {})
        value = item.get('value')
        if value:
            records.append({**metric, 'value': float(value[1]), 'timestamp': datetime.fromtimestamp(value[0])})
    return pd.DataFrame(records)


## 예시 1: Node Exporter 상태 (up)
각 노드의 Node Exporter가 살아 있는지 확인합니다.

In [ ]:
query = 'up{job="node-exporter"}'
promql_to_dataframe(run_promql(query))[['instance', 'value', 'timestamp']]


## 예시 2: CPU 사용률
최근 5분 평균 CPU idle 비율을 이용해서 사용률(%)을 계산합니다.

In [ ]:
cpu_query = '100 - (avg by (instance)(rate(node_cpu_seconds_total{mode="idle", job="node-exporter"}[5m])) * 100)'
promql_to_dataframe(run_promql(cpu_query))[['instance', 'value']].rename(columns={'value': 'cpu_usage_percent'})


## 예시 3: 메모리 사용률
총 메모리 대비 사용 중인 메모리를 계산합니다.

In [ ]:
mem_query = '(1 - node_memory_MemAvailable_bytes{job="node-exporter"} / node_memory_MemTotal_bytes{job="node-exporter"}) * 100'
promql_to_dataframe(run_promql(mem_query))[['instance', 'value']].rename(columns={'value': 'memory_usage_percent'})


## 파일에서 PromQL 실행하기
`docs/study/prometheus/queries` 디렉터리에 `.promql` 파일을 저장해 두고 실행할 수 있습니다.

In [ ]:
QUERIES_DIR = Path('docs/study/prometheus/queries')

def run_query_file(filename: str):
    path = QUERIES_DIR / filename
    if not path.exists():
        raise FileNotFoundError(f'Query file not found: {path}')
    query_text = path.read_text(encoding='utf-8').strip()
    print(f'Running PromQL from {path}:
{query_text}')
    return promql_to_dataframe(run_promql(query_text))

# 예시 실행
run_query_file('node_up.promql')
